# This notebook produces a dictionary where the vocabulary found in the Questions in CommonsenseQA are keys and their triplets which are found in ConceptNet are the values

In [1]:
import json_lines
import re
import requests
from tqdm import tqdm_notebook as tqdm
from time import sleep
import time
import multiprocessing

In [2]:
def load_files(which_dataset): #train / dev / test    
    questions = []
    with open(which_dataset + '_qtoken_split.jsonl', 'rb') as f: # opening file in binary(rb) mode    
        for item in json_lines.reader(f):
            questions.append(item)
    return questions

def build_vocab(questions): #what we get from load_files
    regex = r'\b\w+\b'
    vocab = []
    for Q in questions:
        vocab += re.findall(regex, Q['question']['stem'].lower())
        for i in range(3):
            vocab += Q['question']['choices'][i]['text'].lower()
    vocab = list(set(vocab))
    return vocab

# We use the API for ConceptNet to obtain triplets

In [7]:
#get all of one word's triplet
def get_triplets_single(word):

    #This is the API. 
    obj = requests.get('http://api.conceptnet.io/c/en/' + word).json()


    if 'error' not in obj.keys(): #eliminate words not found in ConceptNet
        list_start = []
        list_rels = []
        list_end = []
        list_lang = []

        len_edges = len(obj['edges'])
        for j in range(len_edges):

            list_start.append(obj['edges'][j]['start']['label'])
            list_lang.append(obj['edges'][j]['start']['language'])
            list_end.append(obj['edges'][j]['end']['label'])
            list_rels.append(obj['edges'][j]['rel']['label'])
        #we will extract triplets which are exclusively in english because 'murica
        relations = [(start,rel,end) for start,rel,end,lang in zip(list_start,
                                                                   list_rels,
                                                                   list_end,
                                                                   list_lang) if lang == 'en']
        
        return (word, relations)

In [8]:
#using multithreading
def get_triplets_multithread(vocab):
    
    pool = multiprocessing.Pool(processes=16)
    pool_outputs = pool.map(get_triplets_single,
                            vocab)
    pool.close()
    pool.join()
    output = [x for x in pool_outputs if x is not None]
    return (dict(output))


In [9]:
#main function
def main(which_data):
    data = load_files(which_data)
    vocab = build_vocab(data)
    triplets = get_triplets_multithread(vocab)
    return triplets

In [18]:
#get the triplets
dev_triplets = main('dev')
test_triplets = main('test')
train_triplets = main('train')


In [ ]:
#save the triplets
json.dump(dev_triplets, open("dev_triplets.txt",'w'))
json.dump(test_triplets, open("test_triplets.txt",'w'))
json.dump(train_triplets, open("train_triplets.txt",'w'))


In [ ]:
#load the triplets
temp_dev_triplets = json.load(open("dev_triplets.txt"))
temp_test_triplets = json.load(open("test_triplets.txt"))
temp_train_triplets = json.load(open("train_triplets.txt"))

In [ ]:
#check for any discrepancies
print(dev_triplets == temp_dev_triplets)
print(test_triplets == temp_test_triplets)
print(train_triplets == temp_train_triplets)


# This function queries the API for ConceptNet one by one (instead of multithreading)

In [ ]:

# def get_triplets(vocab):
#     triplet_dictionary = {}
#     for i, word in tqdm(enumerate(vocab)):
        
        
#         #This is the API. 
#         obj = requests.get('http://api.conceptnet.io/c/en/' + word).json()
        
        
#         if 'error' not in obj.keys(): #eliminate words not found in ConceptNet
#             list_start = []
#             list_rels = []
#             list_end = []
#             list_lang = []

#             len_edges = len(obj['edges'])
#             for j in range(len_edges):
                
#                 list_start.append(obj['edges'][j]['start']['label'])
#                 list_lang.append(obj['edges'][j]['start']['language'])
#                 list_end.append(obj['edges'][j]['end']['label'])
#                 list_rels.append(obj['edges'][j]['rel']['label'])
#             #we will extract triplets which are exclusively in english because 'murica
#             relations = [(start,rel,end) for start,rel,end,lang in zip(list_start,
#                                                                        list_rels,
#                                                                        list_end,
#                                                                        list_lang) if lang == 'en']
#             triplet_dictionary[word] = relations
#     return triplet_dictionary